
# مقارنة معايرة المصنفات

المصنفات المعايرة جيدًا هي مصنفات احتمالية يمكن تفسير ناتجها
من :term:`predict_proba` مباشرةً كدرجة ثقة.
على سبيل المثال، يجب أن يقوم المصنف المعاير جيدًا (الثنائي) بتصنيف العينات
بحيث للعينات التي أعطاها قيمة :term:`predict_proba` قريبة
من 0.8، تنتمي بالفعل حوالي 80% إلى الفئة الإيجابية.

في هذا المثال، سنقارن معايرة أربعة نماذج مختلفة: `Logistic_regression`، `gaussian_naive_bayes`،
`Random Forest Classifier <forest>` و `Linear SVM
<svm_classification>`.


المؤلفون: مطوري سكايلرن
معرف الترخيص: BSD-3-Clause



In [ ]:
#
# مجموعة البيانات
# -------
#
# سنستخدم مجموعة بيانات تصنيف ثنائية اصطناعية مع 100,000 عينة
# و20 ميزة. من 20 ميزة، فقط 2 هي معلوماتية، 2 هي
# مكررة (تركيبات عشوائية من الميزات المعلوماتية) والباقي
# 16 غير معلوماتية (أرقام عشوائية).
#
# من 100,000 عينة، سيتم استخدام 100 لتدريب النماذج والباقي
# للاختبار. لاحظ أن هذا التقسيم غير عادي إلى حد ما: الهدف هو الحصول
# منحنيات معايرة مستقرة لتقديرات النماذج التي قد تكون عرضة للخطر
# الإفراط في التحديد. في الممارسة العملية، يجب استخدام التحقق المتقاطع مع المزيد
# تقسيمات متوازنة ولكن هذا من شأنه أن يجعل رمز هذا المثال أكثر تعقيدًا
# للمتابعة.

from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibrationDisplay
from sklearn.svm import LinearSVC
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

X, y = make_classification(
    n_samples=100_000, n_features=20, n_informative=2, n_redundant=2, random_state=42
)

train_samples = 100  # Samples used for training the models
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    shuffle=False,
    test_size=100_000 - train_samples,
)

## منحنيات المعايرة

أدناه، نقوم بتدريب كل من النماذج الأربعة باستخدام مجموعة التدريب الصغيرة، ثم
رسم منحنيات المعايرة (المعروفة أيضًا باسم الرسوم البيانية الموثوقية) باستخدام
الاحتمالات المتوقعة لمجموعة اختبار البيانات. يتم إنشاء منحنيات المعايرة
عن طريق تجميع الاحتمالات المتوقعة، ثم رسم متوسط الاحتمالية المتوقعة في كل صندوق مقابل
التردد الملاحظ ('نسبة الإيجابيات'). أسفل منحنى المعايرة، نرسم مخططًا تاريخيًا يُظهر
توزيع الاحتمالات المتوقعة أو على وجه التحديد،
عدد العينات في كل صندوق من الاحتمالات المتوقعة.



In [ ]:
class NaivelyCalibratedLinearSVC(LinearSVC):
    """LinearSVC with `predict_proba` method that naively scales
    `decision_function` output."""

    def fit(self, X, y):
        super().fit(X, y)
        df = self.decision_function(X)
        self.df_min_ = df.min()
        self.df_max_ = df.max()

    def predict_proba(self, X):
        """Min-max scale output of `decision_function` to [0,1]."""
        df = self.decision_function(X)
        calibrated_df = (df - self.df_min_) / (self.df_max_ - self.df_min_)
        proba_pos_class = np.clip(calibrated_df, 0, 1)
        proba_neg_class = 1 - proba_pos_class
        proba = np.c_[proba_neg_class, proba_pos_class]
        return proba

In [ ]:
# تحديد المصنفات التي سيتم مقارنتها في الدراسة.
#
# لاحظ أننا نستخدم نسخة من نموذج الانحدار اللوجستي يمكنها
# ضبط معامل التنظيم تلقائيًا.
#
# لمقارنة عادلة، يجب أن نقوم ببحث فرط المعلمات لجميع
# المصنفات ولكننا لا نفعل ذلك هنا من أجل الحفاظ على رمز المثال
# مختصر وسريع للتنفيذ.
lr = LogisticRegressionCV(
    Cs=np.logspace(-6, 6, 101), cv=10, scoring="neg_log_loss", max_iter=1_000
)
gnb = GaussianNB()
svc = NaivelyCalibratedLinearSVC(C=1.0)
rfc = RandomForestClassifier(random_state=42)

clf_list = [
    (lr, "Logistic Regression"),
    (gnb, "Naive Bayes"),
    (svc, "SVC"),
    (rfc, "Random forest"),
]

In [ ]:
fig = plt.figure(figsize=(10, 10))
gs = GridSpec(4, 2)
colors = plt.get_cmap("Dark2")

ax_calibration_curve = fig.add_subplot(gs[:2, :2])
calibration_displays = {}
markers = ["^", "v", "s", "o"]
for i, (clf, name) in enumerate(clf_list):
    clf.fit(X_train, y_train)
    display = CalibrationDisplay.from_estimator(
        clf,
        X_test,
        y_test,
        n_bins=10,
        name=name,
        ax=ax_calibration_curve,
        color=colors(i),
        marker=markers[i],
    )
    calibration_displays[name] = display

ax_calibration_curve.grid()
ax_calibration_curve.set_title("Calibration plots")

# إضافة مخطط تاريخي
grid_positions = [(2, 0), (2, 1), (3, 0), (3, 1)]
for i, (_, name) in enumerate(clf_list):
    row, col = grid_positions[i]
    ax = fig.add_subplot(gs[row, col])

    ax.hist(
        calibration_displays[name].y_prob,
        range=(0, 1),
        bins=10,
        label=name,
        color=colors(i),
    )
    ax.set(title=name, xlabel="Mean predicted probability", ylabel="Count")

plt.tight_layout()
plt.show()

## تحليل النتائج

:class:`~sklearn.linear_model.LogisticRegressionCV` يعيد تنبؤات معايرة جيدة
على الرغم من صغر حجم مجموعة التدريب: منحنى المعايرة الخاص به هو الأقرب إلى القطر بين النماذج الأربعة.

يتم تدريب الانحدار اللوجستي عن طريق تقليل الخسارة اللوجستية التي هي قاعدة صارمة
درجة التهديف: في حدود بيانات التدريب اللانهائية، يتم تقليل قواعد التسجيل الصارمة
بواسطة النموذج الذي يتنبأ بالاحتمالات الشرطية الحقيقية. هذا (النموذج الافتراضي)
سيكون معايرًا تمامًا. ومع ذلك، فإن استخدام قاعدة تسجيل صارمة كهدف للتدريب ليس
كافيًا لضمان نموذج معاير جيدًا بحد ذاته: حتى مع
مجموعة تدريب كبيرة جدًا، يمكن أن يكون الانحدار اللوجستي معايرًا بشكل سيء، إذا
كان منظمًا بشكل مفرط أو إذا كان اختيار الميزات المدخلة يجعل هذا النموذج
غير محدد (على سبيل المثال، إذا كان حد القرار الحقيقي لمجموعة البيانات هو
دالة غير خطية للغاية للميزات المدخلة).

في هذا المثال، تم الاحتفاظ بمجموعة التدريب صغيرة عمدًا. في هذا
الإعداد، يمكن أن يؤدي تقليل الخسارة اللوجستية إلى نماذج معايرة بشكل سيء
بسبب الإفراط في التحديد. لتخفيف ذلك، تم تكوين فئة
:class:`~sklearn.linear_model.LogisticRegressionCV` لتعديل
معامل التنظيم C أيضًا لتقليل الخسارة اللوجستية عبر التحقق المتقاطع الداخلي
للعثور على أفضل حل وسط لهذا النموذج في
إعداد مجموعة التدريب الصغيرة.

بسبب حجم مجموعة التدريب المحدود وعدم وجود ضمان لـ
المعايرة الجيدة، نلاحظ أن منحنى المعايرة لنموذج الانحدار اللوجستي قريب ولكن ليس تمامًا على القطر. يمكن تفسير شكل منحنى المعايرة لهذا النموذج على أنه
غير واثق: الاحتمالات المتوقعة قريبة جدًا من 0.5
مقارنة بنسبة الفئة الإيجابية الحقيقية.

جميع الطرق الأخرى تخرج احتمالات أقل معايرة:

* :class:`~sklearn.naive_bayes.GaussianNB` تميل إلى دفع الاحتمالات إلى 0
  أو 1 (انظر المخطط التاريخي) على هذه المجموعة البيانات المحددة (الثقة المفرطة). هذا هو
بشكل أساسي لأن معادلة خوارزمية بايز الساذجة توفر تقديرًا صحيحًا فقط
الاحتمالات عندما يفترض أن الميزات مشروطة
مستقل [2]_. ومع ذلك، يمكن أن تكون الميزات مترابطة وهذا هو الحال
مع هذه المجموعة البيانات، والتي تحتوي على 2 ميزات تم إنشاؤها كتركيبات خطية
عشوائية من الميزات المعلوماتية. يتم حساب هذه الميزات المترابطة بشكل فعال
'عد مرتين'، مما يؤدي إلى دفع الاحتمالات المتوقعة نحو 0 و1 [3]_. لاحظ، مع ذلك، أن تغيير البذرة
المستخدمة لتوليد مجموعة البيانات يمكن أن يؤدي إلى نتائج مختلفة على نطاق واسع لمقدّر خوارزمية بايز الساذجة.

* :class:`~sklearn.svm.LinearSVC` ليس مصنفًا احتماليًا طبيعيًا.
  لتفسير تنبؤاته كاحتمالية، قمنا بتوسيع نطاق ناتج
  من :term:`decision_function` إلى [0، 1] عن طريق تطبيق التوسيع النطاق في
  فئة NaivelyCalibratedLinearSVC المحددة أعلاه. يظهر هذا
  المصنف منحنى معايرة على شكل حرف S على هذه البيانات:
  تنبؤات أكبر من 0.5 تقابل العينات مع حتى أكبر
  نسبة الفئة الإيجابية الفعالة (فوق القطر)، في حين أن التنبؤات
  أقل من 0.5 تقابل حتى انخفاض نسب الفئة الإيجابية (أسفل القطر). هذه التنبؤات غير الواثقة هي نموذجية للأساليب ذات الهامش الأقصى [1]_.

* :class:`~sklearn.ensemble.RandomForestClassifier`'s prediction histogram
  يظهر قمم عند تقريبًا 0.2 و0.9 احتمالية، في حين أن الاحتمالات القريبة من
  0 أو 1 نادرة جدًا. يمكن تفسير هذا على النحو التالي [1]_:
  "الطرق مثل التجميع والغابات العشوائية التي متوسط
  تنبؤات من مجموعة من النماذج الأساسية يمكن أن يكون لها صعوبة في إجراء
  التنبؤات بالقرب من 0 و1 لأن التباين في النماذج الأساسية
  سيؤدي إلى تحيز التنبؤات التي يجب أن تكون قريبة من الصفر أو الواحد بعيدًا عن هذه
  القيم. لأن التنبؤات مقيدة بالمدى [0، 1]، فإن الأخطاء
  الناجمة عن التباين تميل إلى أن تكون أحادية الجانب بالقرب من الصفر والواحد. على سبيل المثال، إذا
  كان على النموذج التنبؤ بـ p = 0 لعينة، فإن الطريقة الوحيدة للتجميع هي
  إذا تنبأت جميع الأشجار المجمّعة بصفر. إذا أضفنا ضوضاء إلى الأشجار التي
  يتم تجميعها، فإن هذه الضوضاء ستؤدي إلى تنبؤ بعض الأشجار
  بقيم أكبر من 0 لهذه العينة، وبالتالي تحريك متوسط التنبؤات
  لمجموعة الأشجار المجمّعة بعيدًا عن الصفر. نلاحظ هذا التأثير بشكل أقوى مع
  الغابات العشوائية لأن الأشجار الأساسية المدربة باستخدام الغابات العشوائية
  لديها تباين عالٍ نسبيًا بسبب تقسيم الميزات الفرعية. "هذا التأثير يمكن أن يجعل الغابات العشوائية غير واثقة. على الرغم من هذا التحيز المحتمل، لاحظ أن
  يتم تدريب الأشجار نفسها عن طريق تقليل معيار جيني أو Entropy
 ، وكلاهما يؤدي إلى تقسيمات تقلل من قواعد التسجيل الصارمة:
  درجة Brier أو الخسارة اللوجستية على التوالي. انظر `دليل المستخدم
  <tree_mathematical_formulation>` لمزيد من التفاصيل. هذا يمكن أن يفسر لماذا
  هذا النموذج يظهر منحنى معايرة جيد بما فيه الكفاية على هذه المجموعة البيانات
  معينة. في الواقع، نموذج الغابة العشوائية ليس أقل ثقة بشكل كبير
  من نموذج الانحدار اللوجستي.

لا تتردد في إعادة تشغيل هذا المثال باستخدام بذور عشوائية مختلفة ومعلمات أخرى
لتوليد مجموعة البيانات لمشاهدة مدى اختلاف مخططات المعايرة. بشكل عام، الانحدار اللوجستي والغابة العشوائية
سيميلان إلى أن يكونا المصنفان الأكثر معايرة، في حين أن SVC سيعرض في كثير من الأحيان
معايرة غير واثقة. غالبًا ما يكون نموذج خوارزمية بايز الساذجة غير معاير جيدًا أيضًا ولكن الشكل العام
لمنحنى المعايرة يمكن أن يختلف على نطاق واسع اعتمادًا على مجموعة البيانات.

أخيرًا، لاحظ أنه بالنسبة لبعض بذور مجموعة البيانات، تكون جميع النماذج معايرة بشكل سيء،
حتى عند ضبط معامل التنظيم كما هو موضح أعلاه. هذا أمر محتوم
عندما يكون حجم التدريب صغيرًا جدًا أو عندما يكون النموذج غير محدد بشدة.

## المراجع

.. [1] [Predicting Good Probabilities with Supervised Learning](https://dl.acm.org/doi/pdf/10.1145/1102351.1102430), A.
       Niculescu-Mizil & R. Caruana, ICML 2005

.. [2] [Beyond independence: Conditions for the optimality of the simple
       bayesian classifier](https://www.ics.uci.edu/~pazzani/Publications/mlc96-pedro.pdf)
       Domingos, P., & Pazzani, M., Proc. 13th Intl. Conf. Machine Learning.
       1996.

.. [3] [Obtaining calibrated probability estimates from decision trees and
       naive Bayesian classifiers](https://citeseerx.ist.psu.edu/doc_view/pid/4f67a122ec3723f08ad5cbefecad119b432b3304)
       Zadrozny, Bianca, and Charles Elkan. Icml. Vol. 1. 2001.

